Analysis of genotypic diversity by tumor location (including Supp. Fig S4)

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import random
import glob
import pyreadr
from operator import itemgetter
import itertools
from itertools import groupby
import seaborn as sns
import matplotlib.ticker as ticker
import sys
import scipy.stats  as stats
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
tumor_roi_dat = pd.read_csv('new_annotation_Feb2021_KW.txt', delimiter = "\t")
protein_dat=pd.read_csv('ERCC_IgG_norm.csv')
nuc_count = pd.read_csv('nuc_count.csv')

protein_dat=protein_dat.rename({'Unnamed: 0': 'Protein'}, axis=1) 

all_rois=list(protein_dat.columns)[1:]

tumor_core_roi_dict={}
excluded_rois=[]
for loc in range(len(tumor_roi_dat)):
    cur_row = tumor_roi_dat.iloc[loc]
    roi = cur_row['ROI']
    roi_num=roi.split('_')[1]
    if len(roi_num)==1:
        roi = 'ROI_0'+roi_num
    #print(cur_row)
    if roi in all_rois:
        key = (cur_row['Tumor_ID'],cur_row['Core_ID'])
        if key[0]==21 or key[0]==8:
            excluded_rois.append(roi)
        else:
            if key in tumor_core_roi_dict:
                tumor_core_roi_dict[key].append(roi)
            else:
                tumor_core_roi_dict[key] = [roi]            
            

tumor_ids=list(tumor_roi_dat['Tumor_ID'])
core_ids=list(tumor_roi_dat['Core_ID'])
fish_cores=list(tumor_roi_dat['FISH_core'])

tumor_core_to_shreya_base={(tumor_ids[i],core_ids[i]):fish_cores[i] for i in range(len(tumor_roi_dat))}
tumor_core_to_shreya={}
for k,v in tumor_core_to_shreya_base.items():
    if k in tumor_core_roi_dict.keys():
        tumor_core_to_shreya[k]=v
all_proteins_cores_shreya=list(tumor_core_to_shreya.values())

tumor_roi_dict={}
for k, rois in tumor_core_roi_dict.items():
    tumor=k[0]
    if tumor in tumor_roi_dict.keys():
        tumor_roi_dict[tumor]=tumor_roi_dict[tumor]+rois
    else:
        tumor_roi_dict[tumor]=rois
            
nuc_count=nuc_count.rename({'Unnamed: 0': 'ROI'}, axis=1) 

tumor_protein_dfs=[]
for tumor,rois in tumor_roi_dict.items():
    cur_df=protein_dat[rois]
    cur_df.index=protein_dat['Protein']
    tot_nuc_count=nuc_count[nuc_count['ROI'].isin(protein_dat[rois])]['nuclei_count'].sum()
    for roi in rois:
        cur_weight=list(nuc_count[nuc_count['ROI']==roi]['nuclei_count'])[0]/tot_nuc_count
        cur_df[roi]=cur_df[roi]*cur_weight 
    cur_df=cur_df.sum(axis=1).to_frame().rename(columns={0:tumor})
    tumor_protein_dfs.append(cur_df)
protein_dat_by_tumor=pd.concat(tumor_protein_dfs,1)

protein_dat_by_tumor=protein_dat_by_tumor.T

In [ ]:
filenames = glob.glob("FISH_data/table*")
dfs_o = {}

for filename in filenames:
    df_o = pd.read_csv(filename)
    shreya_core_num = int(filename.split('_')[2])
    image_num = int(filename.split('_')[3].split('.')[0])
    dfs_o[(shreya_core_num,image_num)]=df_o

    
HI=6
MAX_HI = 50

def geno_hi(geno,df):
    return (df[geno]>=HI)&(df[geno]<=MAX_HI)
def geno_lo(geno,df):
    return (df[geno]>=1)&(df[geno]<HI)

for df_fish in list(dfs_o.values()):
    df_fish['EGFR_amp']=1*(geno_hi('EGFR',df_fish))
    df_fish['PDGFRA_amp']=1*(geno_hi('PDGFRA',df_fish))
    df_fish['CDK4_amp']=1*(geno_hi('CDK4',df_fish))
    
dfs={}    
for key,df in dfs_o.items():
        
    if key[0] not in tumor_core_to_shreya.values():
        continue

    
    df['EGFR_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']>=HI))
    df['EGFR_NOT_CDK4_amp']=1*((df['EGFR']>=HI)&(df['CDK4']<HI))
    df['CDK4_NOT_EGFR_amp']=1*((df['CDK4']>=HI)&(df['EGFR']<HI))
    
    df['Non_amp']=1*((df['EGFR']<HI)&(df['CDK4']<HI))
    
    dfs[key]=df[(df['EGFR']<=MAX_HI)&(df['PDGFRA']<=MAX_HI)&(df['CDK4']<=MAX_HI)]
   

In [ ]:
shreya_to_tumor={v:k[0] for k,v in tumor_core_to_shreya.items()}
core_list=list(set([item[0] for item in dfs.keys()]))
tumor_list = list(set([shreya_to_tumor[core] for core in core_list]))

tumor_image_dict={}
for k in dfs.keys():
    this_tumor=shreya_to_tumor[k[0]]
    if this_tumor in tumor_image_dict.keys():
        tumor_image_dict[this_tumor]=tumor_image_dict[this_tumor]+[k]
    else:
        tumor_image_dict[this_tumor]=[k]
        
dfs_in_tumor={}
for tumor, images in tumor_image_dict.items():
    dfs_in_tumor[tumor]=pd.concat([dfs[image] for image in images])

In [ ]:
all_cells_df=pd.concat([this_tumor_df for this_tumor_df in dfs_in_tumor.values()])

In [ ]:
len(all_cells_df)

In [ ]:
len(all_cells_df[all_cells_df['EGFR_CDK4_amp']==1])

In [ ]:
len(all_cells_df[all_cells_df['EGFR_NOT_CDK4_amp']==1])

In [ ]:
len(all_cells_df[all_cells_df['CDK4_NOT_EGFR_amp']==1])

In [ ]:
len(all_cells_df[all_cells_df['Non_amp']==1])

In [ ]:
shreya_to_core_num={v:k[1] for k,v in tumor_core_to_shreya.items()}

In [ ]:
def get_shannon(key):
    df=dfs[key]
    EGFR_only_amp=len(df[df['EGFR_NOT_CDK4_amp']==1])
    CDK4_only_amp=len(df[df['CDK4_NOT_EGFR_amp']==1])
    dual_amp=len(df[df['EGFR_CDK4_amp']==1])
    no_amp=len(df[df['Non_amp']==1])
    tot_cells=len(df)
    probs=np.array([EGFR_only_amp,CDK4_only_amp,dual_amp,no_amp])/tot_cells
    probs=list(filter(lambda x: x > 0, probs))
    shannon_ind=-sum([prob*np.log(prob) for prob in probs])
    if np.isnan(shannon_ind):
        print ([EGFR_only_amp,CDK4_only_amp,dual_amp,no_amp])
    return shannon_ind

def get_shannon_tumor(tumor):
    this_dict={im:[get_shannon(im),shreya_to_core_num[im[0]]] for im in tumor_image_dict[tumor]}
    this_df=pd.DataFrame.from_dict(this_dict,orient='index').rename(columns={0:'Shannon Index',1:'Core'})
    this_df['Tumor']=tumor
    this_df=this_df.reset_index().drop(['index'],1)
    return this_df

In [ ]:
shannon_dfs=[get_shannon_tumor(tumor) for tumor in tumor_image_dict.keys()]  
shannon_df=pd.concat(shannon_dfs)

In [ ]:
df_locations=pd.read_csv('tumor_locations.csv')
df_locations=df_locations[['Tumor ID','Tumor Revised Location']]

In [ ]:
loc_dict=dict(zip(df_locations['Tumor ID'], df_locations['Tumor Revised Location']))

In [ ]:
location_list=[]
shannon_tumors=list(shannon_df['Tumor'])
for tumor in shannon_tumors:
    location_list.append(loc_dict[tumor])

In [ ]:
shannon_df['Location']=location_list
shannon_df2=shannon_df.dropna()
shannon_df2=shannon_df2.replace({'Corpus callosum':'Corpus\ncallosum'},regex=True)
shannon_df2=shannon_df2.replace({'Temporo-occipital':'Temporo-\noccipital'},regex=True)

In [ ]:
fig= plt.figure(figsize=(12,8))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 32
ax=sns.stripplot(x="Location", y="Shannon Index",hue="Tumor",data=shannon_df2,size=8,s=15,palette='husl',linewidth=1)
ax.set_ylim([0,1.5])
plt.yticks([0.,0.5,1.0,1.5])
ax.legend_.remove()
plt.xticks(rotation=90)
plt.tight_layout()
#plt.savefig('shannon_catplot_all_tumors_by_location.pdf')
plt.show()